In [7]:
from ProcessBasic import *

In [8]:
def flatten_directional_traffic_df(df: pd.DataFrame) -> pd.DataFrame:
    """
    將具有 MultiIndex 欄位（方向性交通量）的資料表展平為 long-form。
    自動辨識方向欄位與時間欄位（起、迄）。
    
    Parameters:
        df (pd.DataFrame): 原始的 MultiIndex 欄位資料表
        
    Returns:
        pd.DataFrame: 展平後含方向欄的資料表
    """
    if not isinstance(df.columns, pd.MultiIndex):
        raise ValueError("輸入的欄位不是 MultiIndex 結構")

    # 所有第一層欄位
    all_lv0 = df.columns.get_level_values(0).unique()

    # 自動找出方向欄（排除 起 與 迄）
    direction_cols = [col for col in all_lv0 if col not in ['起', '迄']]

    # 自動找出「起」、「迄」欄位（不假設第二層）
    col_起 = next(col for col in df.columns if col[0] == '起')
    col_迄 = next(col for col in df.columns if col[0] == '迄')

    df_list = []

    for direction in direction_cols:
        temp = df[direction].copy()
        temp['方向'] = direction
        temp['起'] = df[col_起]
        temp['迄'] = df[col_迄]
        df_list.append(temp)

    df_flat = pd.concat(df_list, ignore_index=True)

    # 欄位排序
    ordered_cols = ['方向', '起', '迄'] + [col for col in df_flat.columns if col not in ['方向', '起', '迄']]
    return df_flat[ordered_cols]


In [9]:
# Step0 定義資料夾
datafolder = os.path.abspath(os.path.join(get_projectfolderpath(), 'Technical', '06_交通量調查', '02_原始資料'))
volumefolder = os.path.abspath(os.path.join(datafolder, '01_路段交通量'))
speedfolder = os.path.abspath(os.path.join(datafolder, '02_路段旅行速率'))

initialfolder = create_folder(os.path.join(os.getcwd(), '..', '01_資料初步彙整'))
volume_initialfolder = create_folder(os.path.join(initialfolder, '01_路段交通量'))
speed_initialfolder = create_folder(os.path.join(initialfolder, '02_路段旅行速率'))


In [21]:
# Step1 彙整路段交通量資料
files = findfiles(volumefolder,'xlsx')

file = files[0]

In [31]:
surveynumber = os.path.basename(file).split(".")[0] #讀取檔名進行命名(因xlsx內中並沒有可以協助判斷的部分)
surveyplace = os.path.basename(file).split(".")[1]

sheetnamelist = ['平日','假日']
sheetname = sheetnamelist[0]

df = pd.read_excel(file, header=[0,1], sheet_name=sheetname)
df.rename(columns = {'Unnamed: 0_level_0':'起'}, inplace=True)
df.rename(columns = {'Unnamed: 1_level_0':'迄'}, inplace=True)

# 直接從第二層尋找欄位名稱為 1.8 的位置，在此之前才是調查表格
drop_from = next(i for i, col in enumerate(df.columns) if col[1] == 1.8)
df = df.iloc[:, :drop_from] # 使用 iloc 保留 drop_from 之前的所有欄位

df_flat = flatten_directional_traffic_df(df)
df_flat['調查計畫書點位編號'] = surveynumber
df_flat['調查路段'] = surveyplace
df_flat['快慢車道'] = df_flat['方向'].apply(lambda x: '快車道' if '快車道' in x else '慢車道')
df_flat['方向'] = df_flat['方向'].str.split('-').str[0]
df_flat['平假日'] = sheetname
for column in ['平假日', '方向', '快慢車道', '調查路段', '調查計畫書點位編號']:
    df_flat = move_column(df = df_flat, column_name = column, insert_index=0)


In [ ]:
df

In [ ]:
df_flat[df_flat['方向'] == '往西(南)']

In [ ]:
# Step2 彙整路段旅行速率資料